v2.1a - uses model v2.1 for running many parameters of strategies and compare returns and sharpes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

test = pd.DataFrame(columns=['ma_fast', 'ma_slow', 'bb_len', 'bb_mult', 'final_equity'])
ohlcv = yf.download('BA', start='2015-01-01', progress=False)

for x in range(49,51):
    for y in range(2,101):
        if x<y:
            for i in range(11,40):
                for j in range(1,26):
                    # ---------------- PARAMETERS ----------------
                    ticker = "BA"
                    start_date = "2015-01-01"

                    # Bollinger
                    bb_length = j
                    bb_mult = round(i/10, 1)

                    # MA
                    ma_fast = x
                    ma_slow = y

                    # volatility regime
                    delta_ma_window = 30
                    high_delta = 1.2

                    # ---------------- DATA ----------------
                    df = ohlcv.copy()
                    df['PRICE'] = df['Close']

                    # ---------------- INDICATORS ----------------

                    # Bollinger
                    df['mid'] = df['PRICE'].rolling(bb_length, min_periods=1).mean()
                    df['std'] = df['PRICE'].rolling(bb_length, min_periods=1).std()
                    df['upper'] = df['mid'] + bb_mult * df['std']
                    df['lower'] = df['mid'] - bb_mult * df['std']

                    # Moving averages
                    df['MA_fast'] = df['PRICE'].rolling(ma_fast, min_periods=1).mean()
                    df['MA_slow'] = df['PRICE'].rolling(ma_slow, min_periods=1).mean()

                    # ---------------- STRATEGY RETURNS ----------------

                    # Bollinger band
                    df['signal_bb'] = np.where(df['PRICE'] > df['upper'], 1,
                                        np.where(df['PRICE'] < df['mid'], 0, np.nan))
                    df['signal_bb'] = df['signal_bb'].ffill().fillna(0)
                    df['pos_bb'] = df['signal_bb'].shift(1).fillna(0)
                    df['ret_bb'] = df['pos_bb'] * df['PRICE'].pct_change().fillna(0)
                    
                    df['equity_bb'] = (1 + df['ret_bb']).cumprod()

                    # MA crossover trend
                    df['signal_ma'] = np.where(df['MA_fast'] > df['MA_slow'], 1, 0)
                    df['pos_ma'] = df['signal_ma'].shift(1).fillna(0)
                    df['ret_ma'] = df['pos_ma'] * df['PRICE'].pct_change().fillna(0)
                    
                    df['equity_ma'] = (1 + df['ret_ma']).cumprod()

                    # hold benchmark
                    df['ret_hold'] = df['PRICE'].pct_change().fillna(0)
                    df['equity_hold'] = (1 + df['ret_hold']).cumprod()

                    # ---------------- VOLATILITY ----------------
                    df['delta'] = df['PRICE'].diff()
                    df['abs_delta'] = df['delta'].abs()
                    df['delta_ma'] = df['abs_delta'].rolling(delta_ma_window, min_periods=1).mean()

                    df['regime'] = np.where(df['abs_delta'] > (high_delta * df['delta_ma']), 'bb', 'ma')

                    # ---------------- PORTFOLIO ----------------
                    df['ret_regime'] = np.where(df['regime'] == 'bb', df['ret_bb'], df['ret_ma'])

                    df['portfolio_equity'] = (1 + df['ret_regime']).cumprod()

                    # ----------------- SAVE RETURNS AND SHARPE ---------------
                    test.loc[len(test)] = [x, y, j, round(i/10, 1), df['portfolio_equity'].iloc[-1]]

                    # ---------------- REPORT ----------------
                    # def sharpe(returns, periods=252):
                    #     if returns.std() == 0:
                    #         return 0.0
                    #     return (returns.mean() * periods) / (returns.std() * np.sqrt(periods))

                    # def max_drawdown(equity):
                    #     roll_max = equity.cummax()
                    #     dd = (equity - roll_max) / roll_max
                    #     return dd.min()

                    # total_returns = df['ret_regime'].fillna(0)
                    # combined_sharpe = sharpe(total_returns)
                    # combined_mdd = max_drawdown(df['portfolio_equity'])
                    # combined_cagr = df['portfolio_equity'].iloc[-1] ** (252/len(df)) - 1

                    # print("------------------------------------------")
                    # print(f" STRATEGY REPORT - {ticker} - Len:{bb_length} Mult:{bb_mult}")
                    # print("------------------------------------------")
                    # print(f" Sharpe Ratio:        {combined_sharpe:.3f}")
                    # print(f" Max Drawdown:        {combined_mdd:.2%}")
                    # print(f" CAGR:                {combined_cagr:.2%}")
                    # print(f" Final Equity:        {df['portfolio_equity'].iloc[-1]:.2f}x")
                    # print("------------------------------------------")
                    # print(f" Hold Equity:         {df['equity_hold'].iloc[-1]:.2f}x")

                    # # ---------------- PLOTTING (3 panels) ----------------
                    # fig, axes = plt.subplots(1, 3, figsize=(22, 6))

                    # axes[2].sharey(axes[1])

                    # # Ticker price
                    # axes[0].plot(df['PRICE'], label='Price')
                    # axes[0].set_title(f'{ticker} Price')
                    # axes[0].legend()

                    # # Portfolio curve
                    # axes[1].plot(df['portfolio_equity'], label='Regime Portfolio Equity', linewidth=2, color='black')
                    # axes[1].set_title('Regime-Based Portfolio')
                    # axes[1].legend()

                    # # Strategy equity curves
                    # axes[2].plot(df['equity_bb'], label='Bollinger Band')
                    # axes[2].plot(df['equity_ma'], label='Moving Average')
                    # axes[2].set_title('Strategy Equity Curves')
                    # axes[2].legend()

                    # plt.tight_layout()
                    # plt.show()

: 